## Prepare and read data

In [1]:
import gc
import itertools
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import lightgbm as lgb

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error

In [2]:
pd.set_option("max_columns", 300)
pd.set_option("max_rows", 500)

In [3]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

## Feature Engineering

In [4]:
def get_multi_cat_cols(train_df):
    tmp = train_df.iloc[:1000]
    multi_cols = []
    for c in train_df.columns:
        sep_num = tmp[c].astype(str).fillna("").str.contains(";").sum()
        if sep_num > 10:
            multi_cols.append(c)
    return multi_cols

In [5]:
original_cols = train_df.columns.tolist()

In [6]:
multi_cat_cols = get_multi_cat_cols(train_df)

In [7]:
nume_cols = [
    c for c in list(np.setdiff1d(original_cols, multi_cat_cols))
    if c not in ["Salary", "No"] and "float" in train_df[c].dtype.name
]

In [8]:
cat_cols = [c for c in train_df.columns if c not in multi_cat_cols + nume_cols + ["Salary", "No"]]

In [11]:
len(original_cols), len(cat_cols), len(nume_cols), len(multi_cat_cols)

(128, 65, 40, 21)

### Multi-category encoding

In [10]:
for c in tqdm(multi_cat_cols):
    binarizer = MultiLabelBinarizer()
    train_multi_srs = train_df[c].map(lambda x: x.split(";") if x is not np.nan else [])
    test_multi_srs = test_df[c].map(lambda x: x.split(";") if x is not np.nan else [])
    train_arr = binarizer.fit_transform(train_multi_srs)
    test_arr = binarizer.transform(test_multi_srs)
    feat_cols = [f"ohe_{c}_{val}" for val in binarizer.classes_]
    train_feat_df = pd.DataFrame(train_arr, columns=feat_cols, dtype=np.int8)
    test_feat_df = pd.DataFrame(test_arr, columns=feat_cols, dtype=np.int8)
    train_feat_df[f"sum_answer_{c}"] = (train_df[c].str.count(";") + 1).fillna(-1).astype(np.int8)
    test_feat_df[f"sum_answer_{c}"] = (test_df[c].str.count(";") + 1).fillna(-1).astype(np.int8)
    train_df = pd.concat([train_df, train_feat_df], axis=1)
    test_df = pd.concat([test_df, test_feat_df], axis=1)

### Simple-category encoding

In [12]:
for c in cat_cols:
    train_df[c], uniques = pd.factorize(train_df[c], sort=True)
    test_df[c] = uniques.get_indexer(test_df[c])

### Reduce data memory

In [13]:
for c in nume_cols:
    if train_df[c].nunique() > 1000:
        train_df[c] = train_df[c].astype(np.float32)
        test_df[c] = test_df[c].astype(np.float32)
    else:
        train_df[c] = train_df[c].astype(np.float16)
        test_df[c] = test_df[c].astype(np.float16)

In [14]:
for c in cat_cols:
    if train_df[c].max() > 32767:
        train_df[c] = train_df[c].astype(np.int32)
        test_df[c] = test_df[c].astype(np.int32)
    else:
        train_df[c] = train_df[c].astype(np.int16)
        test_df[c] = test_df[c].astype(np.int16)            

In [15]:
train_df.shape, test_df.shape

((33857, 465), (11259, 464))

### Change Category Name

In [16]:
train_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in train_df.columns]
test_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in test_df.columns]

### Aggregation

In [17]:
def get_importance_cols(use_num=50):
    # basicなモデルのimportanceを読み込み
    importance_df = pd.read_csv("../data/importance/003_importance.csv")
    imp_feats = importance_df["feature"].iloc[:use_num].tolist()
    return imp_feats

In [21]:
imp_feats = get_importance_cols(use_num=50)
imp_cat_cols = [c for c in imp_feats if c in cat_cols or c[:4] == "ohe_"]
imp_nume_cols = [c for c in imp_feats if c in nume_cols or c[:4] == "ohe_"]

In [22]:
len(imp_cat_cols), len(imp_nume_cols)

(32, 15)

In [23]:
all_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)
for cat_col in tqdm(imp_cat_cols):
    for nume_col in imp_nume_cols:
        # one-hotは同じカテゴリの場合がある
        if cat_col == nume_col:
            continue
        all_df[f"agg_mean_{cat_col}_{nume_col}"] = \
            all_df.groupby(cat_col)[nume_col].transform("mean").astype(np.float32)
        all_df[f"agg_std_{cat_col}_{nume_col}"] = \
            all_df.groupby(cat_col)[nume_col].transform("std").astype(np.float32)
        all_df[f"agg_max_{cat_col}_{nume_col}"] = \
            all_df.groupby(cat_col)[nume_col].transform("max").astype(np.float32)
        all_df[f"agg_min_{cat_col}_{nume_col}"] = \
            all_df.groupby(cat_col)[nume_col].transform("min").astype(np.float32)
        all_df[f"diff_agg_mean_{cat_col}_{nume_col}"] = \
            all_df[nume_col] - all_df[f"agg_mean_{cat_col}_{nume_col}"]
        all_df[f"rel_agg_mean_{cat_col}_{nume_col}"] = \
            all_df[nume_col] / (1 + all_df[f"agg_mean_{cat_col}_{nume_col}"])
train_df = all_df.iloc[:len(train_df)].reset_index(drop=True)
test_df = all_df.iloc[len(train_df):].reset_index(drop=True)
del all_df
gc.collect()

58

In [24]:
use_cols = [c for c in train_df.columns if c not in multi_cat_cols + ["Salary", "No"]]

In [25]:
folds = KFold(n_splits=5, random_state=2020, shuffle=True)
fold_idx_list = [(trn_idx, val_idx) for trn_idx, val_idx in folds.split(train_df, train_df["Salary"])]

### Target Encoding

In [32]:
# te_cols = [c for c in use_cols if "agg_" not in c and train_df[c].nunique() >= 5]
te_cols = [c for c in train_df.columns if c in cat_cols]

In [34]:
for c in tqdm(te_cols):
    new_col = "te_" + c
    train_df[new_col] = 0
    test_df[new_col] = 0
    for trn_idx, val_idx in fold_idx_list:
        mean_val = train_df.loc[trn_idx].groupby(c)["Salary"].mean().astype(np.float32)
        train_df.loc[val_idx, new_col] = train_df.loc[val_idx, c].map(mean_val)
        test_df.loc[:, new_col] += test_df.loc[:, c].map(mean_val) / len(fold_idx_list)
    train_df[new_col] = train_df[new_col].astype(np.float32)
    test_df[new_col] = test_df[new_col].astype(np.float32)

In [36]:
# multiple target encoding
multi_te_cols = [c for c in train_df.columns if c in cat_cols or c[:4] == "ohe_"]
imp_feats = get_importance_cols(use_num=30)
multi_te_cols = [c for c in multi_te_cols if c in imp_feats]
combi_multi_te_cols = list(itertools.combinations(multi_te_cols, 2))

In [38]:
for col_a, col_b in tqdm(combi_multi_te_cols):
    new_col = "te_" + col_a + "__" + col_b
    train_df[new_col] = 0
    test_df[new_col] = 0
    train_df["tmp"] = train_df[col_a].fillna("").astype(str) + train_df[col_b].fillna("").astype(str)
    test_df["tmp"] = test_df[col_a].fillna("").astype(str) + test_df[col_b].fillna("").astype(str)
    for trn_idx, val_idx in fold_idx_list:
        mean_val = train_df.loc[trn_idx].groupby("tmp")["Salary"].mean().astype(np.float32)
        train_df.loc[val_idx, new_col] = train_df.loc[val_idx, "tmp"].map(mean_val)
        test_df.loc[:, new_col] += test_df.loc[:, "tmp"].map(mean_val) / len(fold_idx_list)
    train_df[new_col] = train_df[new_col].astype(np.float32)
    test_df[new_col] = test_df[new_col].astype(np.float32)
    del train_df["tmp"], test_df["tmp"]
    gc.collect()

In [39]:
use_cols = [c for c in train_df.columns if c not in multi_cat_cols + ["Salary", "No"]]
print(len(use_cols))

3531


## Training

In [40]:
lgb_params = {
            'objective': 'regression',
            "metric": "rmse",
            "verbosity": -1,
            "boosting": "gbdt",
            'learning_rate': 0.05,
            'num_leaves': 64,
            'min_data_in_leaf': 80, 
            'max_depth': 4,
            "bagging_freq": 5,
            "bagging_fraction": 0.8,
            "lambda_l1": 0.5,
            "lambda_l2": 0.5,
            "feature_fraction": 0.1,
            "seed": 2020,
            "num_threads": -1,
            "max_bins": 30
}

In [41]:
importances = pd.DataFrame()
oof_preds = np.zeros(len(train_df))
models = []

for fold_i, (trn_idx, val_idx) in enumerate(fold_idx_list):
    print(f"Fold {fold_i+1}")
    train_dataset = lgb.Dataset(
        train_df.loc[trn_idx, use_cols],
        label = train_df.loc[trn_idx, "Salary"]
    )
    valid_dataset = lgb.Dataset(
        train_df.loc[val_idx, use_cols],
        label = train_df.loc[val_idx, "Salary"]
    )
    model = lgb.train(
                lgb_params,
                train_dataset,
                3000,
                valid_sets = [train_dataset, valid_dataset],
                verbose_eval=100,
                early_stopping_rounds = 100,
                #feval = eval_f1,
                #callbacks = [log_callback],
    )
    imp_df = pd.DataFrame()
    imp_df['feature'] = use_cols
    imp_df['gain'] = model.feature_importance(importance_type="gain")
    importances = pd.concat([importances, imp_df], axis=0, sort=False)
    
    oof_preds[val_idx] = model.predict(train_df.loc[val_idx, use_cols])
    models.append(model)

Fold 1
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 20796.9	valid_1's rmse: 21673.7
[200]	training's rmse: 19668.3	valid_1's rmse: 21116.3
[300]	training's rmse: 19044.1	valid_1's rmse: 20895.7
[400]	training's rmse: 18550.8	valid_1's rmse: 20751.5
[500]	training's rmse: 18140.1	valid_1's rmse: 20685.7
[600]	training's rmse: 17760.2	valid_1's rmse: 20628.6
[700]	training's rmse: 17447	valid_1's rmse: 20624.9
[800]	training's rmse: 17132.9	valid_1's rmse: 20598.8
[900]	training's rmse: 16853.6	valid_1's rmse: 20566.4
[1000]	training's rmse: 16583.2	valid_1's rmse: 20560.3
[1100]	training's rmse: 16327.6	valid_1's rmse: 20548.3
[1200]	training's rmse: 16078.9	valid_1's rmse: 20548.4
Early stopping, best iteration is:
[1129]	training's rmse: 16249.5	valid_1's rmse: 20537.6
Fold 2
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 20778.3	valid_1's rmse: 21771.6
[200]	training's rmse: 19659.2	valid_1's rmse: 21088.4


In [43]:
oof_score = np.sqrt(mean_squared_error(train_df["Salary"], oof_preds))
oof_score

20312.330026819403

In [44]:
importances.groupby("feature")["gain"].mean().sort_values(ascending=False).reset_index().iloc[:300]

,feature,gain
0,te_Country__YearsCodingProf,4.448626e+13
1,te_Country__Employment,2.135811e+13
2,te_Country__Age,2.116322e+13
3,te_Country__ohe_DevType_Student,1.960813e+13
4,te_Country__SalaryType,1.609062e+13
5,te_YearsCodingProf__Currency,1.447356e+13
6,te_Country__YearsCoding,1.440868e+13
7,te_SalaryType__CurrencySymbol,1.440829e+13
8,te_YearsCodingProf__CurrencySymbol,1.371565e+13
9,te_Country__ohe_DevType_Engineering_manager,8.650644e+12


## Prediction

In [45]:
test_pred = np.zeros(len(test_df))

for model in models:
    test_pred += model.predict(test_df[use_cols]) / len(models)

In [46]:
test_pred

array([ 66563.44051278,  90402.52214667,  84160.00187695, ...,
        91410.89121156,  81970.64371795, 118028.1532839 ])

In [47]:
sub_df = pd.read_csv("../input/submit.csv")

In [48]:
sub_df["Salary"] = test_pred

In [49]:
sub_df.to_csv("../predict/013_clean_code_20312.csv", index=False)